# DataLoader 스모크 & 처리량 측정

In [1]:
import os

# 현재 작업 디렉토리 확인
print("현재 작업 디렉토리:", os.getcwd())

# 노트북이 위치한 디렉토리에서 프로젝트 루트 디렉토리로 이동
# 'notebooks/EDA' 에서 'AI_Lab/computer-vision-competition-1SEN' 으로 이동
os.chdir("../../") 

# 작업 디렉토리 변경 후 다시 확인
print("변경 후 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN/notebooks/EDA
변경 후 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [3]:
import time, torch                                   # 시간 측정, PyTorch
import json                                          # JSON 파일 처리
from torch.utils.data import DataLoader              # 데이터 배치 로더
from src.data.dataset import DocTypeDataset          # 커스텀 데이터셋 클래스
from src.data.transforms import get_train_transforms # 학습용 변환 함수

CFG = json.load(open("configs/data_config.json"))    # 데이터 설정 JSON 로드

# 학습 데이터셋 인스턴스 생성
ds = DocTypeDataset(
    CFG["paths"]["train_csv"],                       # 학습 CSV 경로
    CFG["paths"]["train_root"],                      # 학습 이미지 루트
    mode="train",                                    # 모드(train)
    transform=get_train_transforms(                  # 학습용 변환
        CFG["loader"]["img_size"],                   # 이미지 크기
        CFG["augs"]                                  # 증강 설정
    )
)

# DataLoader 인스턴스 생성
dl = DataLoader(
    ds,                                              # 데이터셋 객체
    batch_size=CFG["loader"]["batch_size"],          # 배치 크기
    shuffle=True,                                    # 데이터 셔플
    num_workers=CFG["loader"]["num_workers"],        # 워커 수
    pin_memory=CFG["loader"]["pin_memory"],          # 핀 메모리 사용
    prefetch_factor=CFG["loader"]["prefetch_factor"],# 프리페치 배치 수
    persistent_workers=CFG["loader"]["persistent_workers"]  # 워커 유지 여부
)

# 100배치 처리 속도 측정
start = time.time()                                  # 시작 시각 기록
n = 0                                                # 처리된 이미지 수 카운터
# DataLoader 순회
for i, (x, y) in enumerate(dl):
    n += len(x)                                      # 배치 내 이미지 개수 누적
    # 100배치 이상 처리 시 종료
    if i >= 100:
        break

elapsed = time.time() - start                        # 경과 시간 계산
print(f"imgs/sec ≈ {n/elapsed:.2f}")                 # 초당 처리 이미지 수 출력

imgs/sec ≈ 35.64
